hosts 파일
```
192.168.10.1    c01
192.168.10.2    c02
192.168.10.3    c03
192.168.10.4    c04
192.168.10.5    c05
192.168.10.6    c06
192.168.10.7    c07
```

absible (http://docs.ansible.com/ansible/)

설치

```
sudo apt-get install software-properties-common
sudo apt-add-repository ppa:ansible/ansible
sudo apt-get update
sudo apt-get install ansible
```

초기 SSH 접속시 Key 등록 여부 묻지 않도록 설정

```
/etc/ansible/ansible.cfg

...
# uncomment this to disable SSH key host checking
host_key_checking = False
...

```

초기 root 계정으로 암호 자동입력하기 위한 설정

```
/etc/ansible/hosts

[cluster]
192.168.10.1
192.168.10.2
192.168.10.3
192.168.10.4
192.168.10.5
192.168.10.6
192.168.10.7

[cluster:vars]
ansible_connection=ssh 
ansible_ssh_user=root     
ansible_ssh_pass=password
```



SSH Key 등록

클러스터안의 각 서버끼리 암호 없이 SSH로 접속하기 위하여 Master의 SSH 키를 클러스터 서버들에 등록합니다.

1. SSH 키 만들기

```
ssh-keygen -t rsa

Generating public/private rsa key pair.
Enter file in which to save the key (/home/comafire/.ssh/id_rsa): <return>
Enter passphrase (empty for no passphrase): <Type the passphrase>

...(생략)
```

Master 서버의 .ssh 디렉토리 안에 id_rsa(개인키), id_rsa.pub(공개키) 가 생성됩니다.

2. SSH 키 교환
클러스터의 각 서버에 생성한 공개키를 복사해 줍니다.

```
scp $HOME/.ssh/id_rsa.pub root@c01:
scp $HOME/.ssh/id_rsa.pub root@c02:

...(생략)
```

3. SSH 키 등록
암호 없이 서버의 접속을 받아들이기 위해 복사된 공개키를 authorized_keys(인증키) 파일에 추가해 줍니다.

```
ssh c01
ehg@c01's password:
cat $HOME/id_rsa.pub >> $HOME/.ssh/authorized_keys

ssh c02
ehg@c02's password:
cat $HOME/id_rsa.pub >> $HOME/.ssh/authorized_keys

...(생략)
```


docker swarm

```
root@c01:~/bdaas/bdaas-spark# docker swarm init --advertise-addr 192.168.10.1
Swarm initialized: current node (hffh10j16mkf2lz96ttf6zvii) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-1z86kig96a3uu1ii7hjjjjgft981z5zhjx1zaexdwwuhqle5o4-1qla9ov4mx2ij49u8sg2wik75 \
    192.168.10.1:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

root@c01:~/bdaas/bdaas-spark# 
```

docker service create --name spark-master --network spark -p 8070:8080 bdaas/spark:2.1.0 /usr/local/spark/bin/spark-class org.apache.spark.deploy.master.Master

docker service create --name spark-worker --replicas 5 --network spark bdaas/spark:2.1.0 /usr/local/spark/bin/spark-class org.apache.spark.deploy.worker.Worker spark://spark-master:7077

root@c01:~/bdaas/bdaas-spark# docker service create --name spark-worker --replicas 5 --network spark bdaas/spark:2.1.0 /usr/local/spark/bin/spark-class org.apache.spark.deploy.worker.Worker spark://10.0.0.3:7077